# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_main = "weather_data.csv"
weather_df = pd.read_csv(weather_main)
weather_main_df = pd.DataFrame(weather_df)
weather_main_df.head()

,Unnamed: 0,city,Lat,lon,max temp,Humidity,Cloudiness,wind speed,country,date
0,0,Hong Kong,22.2855,114.1577,73.76,73,67,5.06,HK,1579662000
1,1,Albany,42.6001,-73.9662,16.88,83,0,3.96,US,1579662000
2,2,Quatre Cocos,-20.2078,57.7625,80.06,86,100,13.94,MU,1579662000
3,3,Cape Town,-33.9258,18.4232,68.61,76,60,2.95,ZA,1579662000
4,4,Novikovo,51.1537,37.8879,35.35,92,99,16.22,RU,1579662000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_main_df[["Lat", "lon"]].astype(float)
hum_percent = weather_main_df["Humidity"].astype(float)

In [4]:
Hum_heatmap = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
heat = gmaps.heatmap_layer(locations, weights=hum_percent,
                          dissipating=False, max_intensity=100,
                          point_radius = 1, opacity=1)
Hum_heatmap.add_layer(heat)
Hum_heatmap


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
max_temp = weather_main_df.loc[weather_main_df["max temp"]<80,:]
low_temp = max_temp.loc[weather_main_df["max temp"]>70,:]
low_temp.head()

max_wind = low_temp.loc[low_temp["wind speed"]<10,:]
max_wind

no_clouds = max_wind.loc[max_wind["Cloudiness"]==0,:]
no_clouds.head()

,Unnamed: 0,city,Lat,lon,max temp,Humidity,Cloudiness,wind speed,country,date
105,105,Moron,-34.6534,-58.6198,75.11,78,0,9.82,AR,1579662000
107,107,Puerto Escondido,15.8500,-97.0667,74.30,70,0,3.71,MX,1579662000
247,247,Champerico,14.3000,-91.9167,75.40,74,0,5.95,GT,1579662000
275,275,Srivardhan,18.0333,73.0167,74.95,59,0,4.72,IN,1579662000
402,402,Ocos,14.5094,-92.1933,75.72,74,0,5.95,GT,1579662000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame()
hotel_name = pd.DataFrame(no_clouds)
hotel_df['city'] = hotel_name['city']
hotel_df['country'] = hotel_name['country']
hotel_df['Lat'] = hotel_name['Lat']
hotel_df['lon'] = hotel_name['lon']
hotel_df.head()

,city,country,Lat,lon
105,Moron,AR,-34.6534,-58.6198
107,Puerto Escondido,MX,15.8500,-97.0667
247,Champerico,GT,14.3000,-91.9167
275,Srivardhan,IN,18.0333,73.0167
402,Ocos,GT,14.5094,-92.1933


In [20]:
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['lon']
    city = row['city']
    country = row['country']
    
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    results = response['results']
    
    try:
        hotel_name = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotel_name}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("-----------------------------------------------------------------")
        

Closest hotel to Moron at -34.6534 , -58.6198 is Frascino.
-----------------------------------------------------------------
Closest hotel to Puerto Escondido at 15.85 , -97.0667 is Hotel Paraiso Escondido.
-----------------------------------------------------------------
Closest hotel to Champerico at 14.3 , -91.9167 is 7 Seas Restaurant.
-----------------------------------------------------------------
Closest hotel to Srivardhan at 18.0333 , 73.0167 is gurukrupa homestay.
-----------------------------------------------------------------
Closest hotel to Ocos at 14.5094 , -92.1933 is LyA Cafeteria y Antojitos.
-----------------------------------------------------------------
Closest hotel to Paysandu at -32.3214 , -58.0756 is Hotel Casagrande.
-----------------------------------------------------------------
Closest hotel to Buin at -33.7333 , -70.75 is Hostal Condell.
-----------------------------------------------------------------
Closest hotel to Ponnani at 10.767000000000001 , 7

In [21]:

hotel_df.head()

,city,country,Lat,lon,Hotel Name
105,Moron,AR,-34.6534,-58.6198,Frascino
107,Puerto Escondido,MX,15.8500,-97.0667,Hotel Paraiso Escondido
247,Champerico,GT,14.3000,-91.9167,7 Seas Restaurant
275,Srivardhan,IN,18.0333,73.0167,gurukrupa homestay
402,Ocos,GT,14.5094,-92.1933,LyA Cafeteria y Antojitos


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lon"]]